# Workflow for a multi-regional energy system

In this application of the FINE framework, a multi-regional energy system is modeled and optimized.

All classes which are available to the user are utilized and examples of the selection of different parameters within these classes are given.

The workflow is structures as follows:
1. Required packages are imported and the input data path is set
2. An energy system model instance is created
3. Commodity sources are added to the energy system model
4. Commodity conversion components are added to the energy system model
5. Commodity storages are added to the energy system model
6. Commodity transmission components are added to the energy system model
7. Commodity sinks are added to the energy system model
8. The energy system model is optimized
9. Selected optimization results are presented


# 1. Import required packages and set input data path

The FINE framework is imported which provides the required classes and functions for modeling the energy system.

In [22]:
import FINE as fn
from getData import getData
import os
cwd = os.getcwd()
data = getData()

%matplotlib inline  
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 2. Create an energy system model instance 

The structure of the energy system model is given by the considered locations, commodities, the number of time steps as well as the hours per time step.

The commodities are specified by a unit (i.e. 'GW_electric', 'GW_H2lowerHeatingValue', 'Mio. t CO2/h') which can be given as an energy or mass unit per hour. Furthermore, the cost unit and length unit are specified.

In [2]:
# locations = {'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7'}
locations = {'es', 'pt', 'fr', 'be', 'lu', 'nl', 'de', 'dk', 'cz', 'pl', 'sk', 'ch', 'at', 'it', 'si', 'hu', 'ro', 'hr', 'ba', 'me', 'rs', 'bg', 'mk', 'gr', 'al', 'ee', 'fi', 'lt', 'lv', 'no', 'se', 'uk', 'ie'}

commodityUnitDict = {'electricity': r'GW$_{el}$', 'methane': r'GW$_{CH_{4},LHV}$', 'biogas': r'GW$_{biogas,LHV}$',
                     'CO2': r'Mio. t$_{CO_2}$/h', 'hydrogen': r'GW$_{H_{2},LHV}$'}
commodities = {'electricity', 'hydrogen', 'methane', 'biogas', 'CO2'}
numberOfTimeSteps = 8760
hoursPerTimeStep = 1

In [3]:
esM = fn.EnergySystemModel(locations=locations, commodities=commodities, numberOfTimeSteps=8760, 
                           commodityUnitsDict=commodityUnitDict, hoursPerTimeStep=1, 
                           costUnit='1e9 Euro', lengthUnit='km', verboseLogLevel=0)

In [4]:
CO2_reductionTarget = 1

# 3. Add commodity sources to the energy system model

## 3.1. Electricity sources

### Wind onshore

In [5]:
esM.add(fn.Source(esM=esM, name='Wind (onshore)', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind (onshore), operationRateMax'],
                  capacityMax=data['Wind (onshore), capacityMax'],
                  investPerCapacity=1.1, opexPerCapacity=1.1*0.02, interestRate=0.08,
                  economicLifetime=25))

In [6]:
type(data['Wind (onshore), capacityMax'])

pandas.core.series.Series

Full load hours:

In [35]:
data['Wind (onshore), operationRateMax'].max()

es     68.464513
pt     11.780070
fr    121.949588
be     10.829575
lu      0.734023
nl     14.896005
de     97.646604
dk     18.582013
cz     10.165080
pl     81.365301
sk      5.194779
ch      1.370339
at      6.785917
it     40.309832
si      0.468821
hu      4.864022
ro      4.794456
hr      6.210808
ba      2.581457
me      0.516498
rs      1.421363
bg      4.367982
mk      0.368496
gr     25.619536
al      2.416273
ee      8.086175
fi     29.329323
lt     15.060887
lv     13.717992
no     12.083263
se     23.994552
uk     92.472990
ie     13.536224
dtype: float64

In [37]:
fn.Component?


### Wind offshore

In [8]:
esM.add(fn.Source(esM=esM, name='Wind (offshore)', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind (offshore), operationRateMax'],
                  capacityMax=data['Wind (offshore), capacityMax'],
                  investPerCapacity=2.3, opexPerCapacity=2.3*0.02, interestRate=0.08,
                  economicLifetime=20))

Full load hours:

In [9]:
data['Wind (offshore), operationRateMax'].sum()

es         0.000000
pt         0.000000
fr         0.000000
be     12893.951068
lu         0.000000
nl     73790.857592
de    131519.843228
dk    122748.518044
cz         0.000000
pl         0.000000
sk         0.000000
ch         0.000000
at         0.000000
it         0.000000
si         0.000000
hu         0.000000
ro         0.000000
hr         0.000000
ba         0.000000
me         0.000000
rs         0.000000
bg         0.000000
mk         0.000000
gr         0.000000
al         0.000000
ee         0.000000
fi         0.000000
lt         0.000000
lv         0.000000
no     15631.914047
se     13495.158885
uk    144228.724482
ie         0.000000
dtype: float64

### PV

In [10]:
esM.add(fn.Source(esM=esM, name='PV', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['PV, operationRateMax'], capacityMax=data['PV, capacityMax'],
                  investPerCapacity=0.65, opexPerCapacity=0.65*0.02, interestRate=0.08,
                  economicLifetime=25))

Full load hours:

In [11]:
data['PV, operationRateMax'].sum()

es     97404.051510
pt     17663.554887
fr    146438.256713
be     11588.065255
lu       797.301653
nl     15981.864885
de    102996.825930
dk     18716.566456
cz     11518.967363
pl     86621.321438
sk      5924.322673
ch      1659.599762
at      8030.780782
it     57643.861616
si       588.149903
hu      5700.426362
ro      5864.540789
hr      8385.924481
ba      3473.776629
me       707.422124
rs      1819.285649
bg      5756.460574
mk       480.776703
gr     40121.335193
al      3327.634466
ee      8087.409695
fi     23953.844394
lt     15362.034890
lv     14175.289574
no      9940.213469
se     21872.110024
uk     92078.547763
ie     13888.284664
dtype: float64

### Exisisting run-of-river hydroelectricity plants

In [12]:
# esM.add(fn.Source(esM=esM, name='Existing run-of-river plants', commodity='electricity',
#                   hasCapacityVariable=True,
#                   operationRateFix=data['Existing run-of-river plants, operationRateFix'], tsaWeight=0.01,
#                   capacityFix=data['Existing run-of-river plants, capacityFix'],
#                   investPerCapacity=0, opexPerCapacity=0.208))

## 3.2. Methane (natural gas and biogas)

### Natural gas

In [ ]:
# esM.add(fn.Source(esM=esM, name='Natural gas purchase', commodity='methane',
#                   hasCapacityVariable=False, commodityCost=0.0331*1e-3))

### Biogas

In [ ]:
# esM.add(fn.Source(esM=esM, name='Biogas purchase', commodity='biogas',
#                   operationRateMax=data['Biogas, operationRateMax'], hasCapacityVariable=False,
#                   commodityCost=0.05409*1e-3))


In [ ]:
## 3.3 CO2

### CO2

The CO2 source is required for the case in which it can be mixed with conventional natural gas.

In [13]:
esM.add(fn.Source(esM=esM, name='CO2 from enviroment', commodity='CO2',
                  hasCapacityVariable=False, commodityLimitID='CO2 limit', yearlyLimit=366*(1-CO2_reductionTarget)))

# 4. Add conversion components to the energy system model

### Combined cycle gas turbine plants

In [ ]:
# esM.add(fn.Conversion(esM=esM, name='CCGT plants (methane)', physicalUnit=r'GW$_{el}$',
#                       commodityConversionFactors={'electricity': 1, 'methane': -1/0.625, 'CO2': 201*1e-6/0.625},
#                       hasCapacityVariable=True,
#                       investPerCapacity=0.65, opexPerCapacity=0.021, interestRate=0.08,
#                       economicLifetime=33))

### New combined cycle gas turbine plants for biogas

In [ ]:
# esM.add(fn.Conversion(esM=esM, name='New CCGT plants (biogas)', physicalUnit=r'GW$_{el}$',
#                       commodityConversionFactors={'electricity':1, 'biogas':-1/0.635},
#                       hasCapacityVariable=True, 
#                       investPerCapacity=0.7, opexPerCapacity=0.021, interestRate=0.08,
#                       economicLifetime=33))

### New combined cycly gas turbines for hydrogen

In [14]:
esM.add(fn.Conversion(esM=esM, name='New CCGT plants (hydrogen)', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':1, 'hydrogen':-1/0.6},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.7, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))

### Electrolyzers

In [15]:
esM.add(fn.Conversion(esM=esM, name='Electroylzers', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.7},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.5, opexPerCapacity=0.5*0.025, interestRate=0.08,
                      economicLifetime=10))

### rSOC

In [ ]:
# capexRSOC=1.5

# esM.add(fn.Conversion(esM=esM, name='rSOEC', physicalUnit=r'GW$_{el}$', linkedConversionCapacityID='rSOC',
#                       commodityConversionFactors={'electricity':-1, 'hydrogen':0.6},
#                       hasCapacityVariable=True, 
#                       investPerCapacity=capexRSOC/2, opexPerCapacity=capexRSOC*0.02/2, interestRate=0.08,
#                       economicLifetime=10))

# esM.add(fn.Conversion(esM=esM, name='rSOFC', physicalUnit=r'GW$_{el}$', linkedConversionCapacityID='rSOC',
#                       commodityConversionFactors={'electricity':1, 'hydrogen':-1/0.6},
#                       hasCapacityVariable=True, 
#                       investPerCapacity=capexRSOC/2, opexPerCapacity=capexRSOC*0.02/2, interestRate=0.08,
#                       economicLifetime=10))

# 5. Add commodity storages to the energy system model

## 5.1. Electricity storage

### Lithium ion batteries

The self discharge of a lithium ion battery is here described as 3% per month. The self discharge per hours is obtained using the equation (1-$\text{selfDischarge}_\text{hour})^{30*24\text{h}} = 1-\text{selfDischarge}_\text{month}$.

In [16]:
esM.add(fn.Storage(esM=esM, name='Li-ion batteries', commodity='electricity',
                   hasCapacityVariable=True, chargeEfficiency=0.95,
                   cyclicLifetime=10000, dischargeEfficiency=0.95, selfDischarge=1-(1-0.03)**(1/(30*24)),
                   chargeRate=1, dischargeRate=1, doPreciseTsaModeling=False,
                   investPerCapacity=0.151, opexPerCapacity=0.002, interestRate=0.08,
                   economicLifetime=22))

## 5.2. Hydrogen storage

### Hydrogen filled salt caverns
The maximum capacity is here obtained by: dividing the given capacity (which is given for methane) by the lower heating value of methane and then multiplying it with the lower heating value of hydrogen.

In [ ]:
# esM.add(fn.Storage(esM=esM, name='Salt caverns (hydrogen)', commodity='hydrogen',
#                    hasCapacityVariable=True, capacityVariableDomain='continuous',
#                    capacityPerPlantUnit=133,
#                    chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
#                    stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (hydrogen), capacityMax'],
#                    investPerCapacity=0.00011, opexPerCapacity=0.00057, interestRate=0.08,
#                    economicLifetime=30))

## 5.3. Methane storage

### Methane filled salt caverns

In [ ]:
# esM.add(fn.Storage(esM=esM, name='Salt caverns (biogas)', commodity='biogas',
#                    hasCapacityVariable=True, capacityVariableDomain='continuous',
#                    capacityPerPlantUnit=443,
#                    chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
#                    stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (methane), capacityMax'],
#                    investPerCapacity=0.00004, opexPerCapacity=0.00001, interestRate=0.08,
#                    economicLifetime=30))

## 5.4 Pumped hydro storage

### Pumped hydro storage

In [ ]:
# esM.add(fn.Storage(esM=esM, name='Pumped hydro storage', commodity='electricity',
#                    chargeEfficiency=0.88, dischargeEfficiency=0.88,
#                    hasCapacityVariable=True, selfDischarge=1-(1-0.00375)**(1/(30*24)),
#                    chargeRate=0.16, dischargeRate=0.12, capacityFix=data['Pumped hydro storage, capacityFix'],
#                    investPerCapacity=0, opexPerCapacity=0.000153))

# 6. Add commodity transmission components to the energy system model

## 6.1. Electricity transmission

### AC cables

In [17]:
# esM.add(fn.LinearOptimalPowerFlow(esM=esM, name='AC cables', commodity='electricity',
#                                   hasCapacityVariable=True, capacityFix=data['AC cables, capacityFix'],
#                                   reactances=data['AC cables, reactances']))

KeyError: 'AC cables, capacityFix'

In [23]:
esM.add(fn.Transmission(esM=esM, name='AC cables', commodity='electricity',
                        
                                  hasCapacityVariable=True, capacityFix=data['AC cables, capacityFix']))

The distances of a component are set to a normalized value of 1.


### DC cables

esM.add(fn.Transmission(esM=esM, name='DC cables', commodity='electricity', losses=data['DC cables, losses'],
                        distances=data['DC cables, distances'],
                        hasCapacityVariable=True, capacityFix=data['DC cables, capacityFix']))

## 6.2 Methane transmission

### Methane pipeline

In [24]:
esM.add(fn.Transmission(esM=esM, name='Pipelines (biogas)', commodity='biogas', 
                        distances=data['Pipelines, distances'],
                        hasCapacityVariable=True, hasIsBuiltBinaryVariable=False, bigM=300,
                        locationalEligibility=data['Pipelines, eligibility'],
                        capacityMax=data['Pipelines, eligibility']*15, sharedPotentialID='pipelines',
                        investPerCapacity=0.000037, investIfBuilt=0.000314,
                        interestRate=0.08, economicLifetime=40))

esM.getComponent('Pipelines (methane)')._mapC['cluster_0_cluster_4'][1]

## 6.3 Hydrogen transmission

### Hydrogen pipelines

In [ ]:
# esM.add(fn.Transmission(esM=esM, name='Pipelines (hydrogen)', commodity='hydrogen',
#                         distances=data['Pipelines, distances'],
#                         hasCapacityVariable=True, hasIsBuiltBinaryVariable=False, bigM=300,
#                         locationalEligibility=data['Pipelines, eligibility'],
#                         capacityMax=data['Pipelines, eligibility']*15, sharedPotentialID='pipelines',
#                         investPerCapacity=0.000177, investIfBuilt=0.00033,
#                         interestRate=0.08, economicLifetime=40))

esM.getComponent('Pipeline (hydrogen)')._mapC

# 7. Add commodity sinks to the energy system model

## 7.1. Electricity sinks

### Electricity demand

In [25]:
esM.add(fn.Sink(esM=esM, name='Electricity demand', commodity='electricity',
                hasCapacityVariable=False, operationRateFix=data['Electricity demand, operationRateFix']))

## 7.2. Hydrogen sinks

### Fuel cell electric vehicle (FCEV) demand

In [ ]:
# FCEV_penetration=0.5
# esM.add(fn.Sink(esM=esM, name='Hydrogen demand', commodity='hydrogen', hasCapacityVariable=False,
#                 operationRateFix=data['Hydrogen demand, operationRateFix']*FCEV_penetration))

## 7.3. CO2 sinks

### CO2 exiting the system's boundary

In [26]:
esM.add(fn.Sink(esM=esM, name='CO2 to enviroment', commodity='CO2',
                hasCapacityVariable=False, commodityLimitID='CO2 limit', yearlyLimit=366*(1-CO2_reductionTarget)))

# 8. Optimize energy system model

All components are now added to the model and the model can be optimized. If the computational complexity of the optimization should be reduced, the time series data of the specified components can be clustered before the optimization and the parameter timeSeriesAggregation is set to True in the optimize call.

In [27]:
esM.cluster(numberOfTypicalPeriods=14)


Clustering time series data with 14 typical periods and 24 time steps per period...
		(31.7319 sec)



In [28]:
esM.optimize(timeSeriesAggregation=True, optimizationSpecs='OptimalityTol=1e-3 cuts=0 method=2')

Time series aggregation specifications:
Number of typical periods:14, number of time steps per periods:24

Declaring sets, variables and constraints for SourceSinkModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(5.9943 sec)

Declaring sets, variables and constraints for ConversionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(1.1429 sec)

Declaring sets, variables and constraints for StorageModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(6.9653 sec)

Declaring sets, variables and constraints for TransmissionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(5.9862 sec)

Declaring shared potential constraint...
		(0.0177 sec)

Declaring commodity balances...
		(11.9522 sec)

Declaring objective function...
		(6.1037 sec)

Academic license - for non-commercial use only
Read LP format model from file C:\Users\r.beer\code\FINE\examples\EU_without_aggregation\tmp4

# 9. Selected results output

Plot locations (GeoPandas required)

In [ ]:
locFilePath = os.path.join(cwd, 'InputData', 'SpatialData','ShapeFiles', 'clusteredRegions.shp')
fig, ax = fn.plotLocations(locFilePath, plotLocNames=True, indexColumn='index')

### Sources and Sink

Show optimization summary

In [33]:
esM.getOptimizationSummary("SourceSinkModel", outputLevel=2).sum(axis=1
                                                                )

Component           Property   Unit           
Electricity demand  operation  [GW$_{el}$*h/a]    4249.609790
PV                  TAC        [1e9 Euro/a]          0.000539
                    capacity   [GW$_{el}$]           0.007293
                    capexCap   [1e9 Euro/a]          0.000444
                    invest     [1e9 Euro]            0.004740
                    operation  [GW$_{el}$*h/a]     512.756294
                    opexCap    [1e9 Euro/a]          0.000095
Wind (onshore)      TAC        [1e9 Euro/a]          0.003379
                    capacity   [GW$_{el}$]           0.027018
                    capexCap   [1e9 Euro/a]          0.002784
                    invest     [1e9 Euro]            0.029720
                    operation  [GW$_{el}$*h/a]    3736.853495
                    opexCap    [1e9 Euro/a]          0.000594
dtype: float64

Plot installed capacities

In [ ]:
fig, ax = fn.plotLocationalColorMap(esM, 'Wind (offshore)', locFilePath, 'index', perArea=True)

Plot operation time series (either one or two dimensional)

In [ ]:
fig, ax = fn.plotOperation(esM, 'Electricity demand', 'cluster_0')

In [ ]:
fig, ax = fn.plotOperationColorMap(esM, 'Electricity demand', 'cluster_0')

### Conversion

Show optimization summary

In [30]:
esM.getOptimizationSummary("ConversionModel", outputLevel=2)

,,,al,at,ba,be,bg,ch,cz,de,dk,ee,...,nl,no,pl,pt,ro,rs,se,si,sk,uk
Component,Property,Unit,,,,,,,,,,,,,,,,,,,,,


In [ ]:
fig, ax = fn.plotOperationColorMap(esM, 'New CCGT plants (biogas)', 'cluster_2')

### Storage

Show optimization summary

In [31]:
esM.getOptimizationSummary("StorageModel", outputLevel=2)

,,,al,at,ba,be,bg,ch,cz,de,dk,ee,...,nl,no,pl,pt,ro,rs,se,si,sk,uk
Component,Property,Unit,,,,,,,,,,,,,,,,,,,,,


In [ ]:
fig, ax = fn.plotOperationColorMap(esM, 'Li-ion batteries', 'cluster_2', 
                                   variableName='stateOfChargeOperationVariablesOptimum')

In [ ]:
fig, ax = fn.plotOperationColorMap(esM, 'Pumped hydro storage', 'cluster_2',
                                  variableName='stateOfChargeOperationVariablesOptimum')

In [ ]:
fig, ax = fn.plotOperationColorMap(esM, 'Salt caverns (biogas)', 'cluster_2',
                                  variableName='stateOfChargeOperationVariablesOptimum')

In [ ]:
fig, ax = fn.plotOperationColorMap(esM, 'Salt caverns (hydrogen)', 'cluster_2',
                                  variableName='stateOfChargeOperationVariablesOptimum')

## Transmission

Show optimization summary

In [32]:
esM.getOptimizationSummary("TransmissionModel", outputLevel=2)

al       at       ba        be  \
AC cables capacity  [GW$_{el}$]     al      NaN      NaN      NaN       NaN   
                                    at      NaN     24.8      NaN       NaN   
                                    ba      NaN      NaN      NaN       NaN   
                                    be      NaN      NaN      NaN       NaN   
                                    bg      NaN      NaN      NaN       NaN   
                                    ch      NaN      2.4      NaN       NaN   
                                    cz      NaN      2.1      NaN       NaN   
                                    de      NaN     18.8      NaN         6   
                                    dk      NaN      NaN      NaN       NaN   
                                    ee      NaN      NaN      NaN       NaN   
                                    es      NaN      NaN      NaN       NaN   
                                    fi      NaN      NaN      NaN       NaN   
                                    fr      NaN      NaN      NaN       7.6   
                                    gr      0.8      NaN      NaN       NaN   
                                    hr      NaN      NaN        4       NaN   
                                    hu      NaN      1.6      NaN       NaN   
                                    ie      NaN      NaN      NaN       NaN   
                                    it        1     10.3      NaN       NaN   
                                    lt      NaN      NaN      NaN       NaN   
                                    lu      NaN      NaN      NaN       0.7   
                                    lv      NaN      NaN      NaN       NaN   
                                    me      0.9      NaN      1.4       NaN   
                                    mk      0.7      NaN      NaN       NaN   
                                    nl      NaN      NaN      NaN      13.5   
                                    no      NaN      NaN      NaN       NaN   
                                    pl      NaN      NaN      NaN       NaN   
                                    pt      NaN      NaN      NaN       NaN   
                                    ro      NaN      NaN      NaN       NaN   
                                    rs      0.9      NaN      3.1       NaN   
                                    se      NaN      NaN      NaN       NaN   
...                                         ...      ...      ...       ...   
          operation [GW$_{el}$*h/a] ba      NaN      NaN      NaN       NaN   
                                    be      NaN      NaN      NaN       NaN   
                                    bg      NaN      NaN      NaN       NaN   
                                    ch      NaN  633.662      NaN       NaN   
                                    cz      NaN  21.8666      NaN       NaN   
                                    de      NaN  8421.26      NaN   7003.07   
                                    dk      NaN      NaN      NaN       NaN   
                                    ee      NaN      NaN      NaN       NaN   
                                    es      NaN      NaN      NaN       NaN   
                                    fi      NaN      NaN      NaN       NaN   
                                    fr      NaN      NaN      NaN    6893.9   
                                    gr    119.2      NaN      NaN       NaN   
                                    hr      NaN      NaN   444.47       NaN   
                                    hu      NaN  516.463      NaN       NaN   
                                    ie      NaN      NaN      NaN       NaN   
                                    it  1594.63  2162.92      NaN       NaN   
                                    lt      NaN      NaN      NaN       NaN   
                                    lu      NaN      NaN      NaN   4831.43   
                                    lv      NaN      NaN      NaN       NaN   
                

Check that the shared capacity of the pipelines are not exceeded

In [ ]:
df=esM.componentModelingDict["TransmissionModel"].capacityVariablesOptimum
df.loc['Pipelines (biogas)']+df.loc['Pipelines (hydrogen)']

Plot installed transmission capacities

In [ ]:
transFilePath = os.path.join(cwd, 'InputData', 'SpatialData','ShapeFiles', 'AClines.shp')

fig, ax = fn.plotLocations(locFilePath, indexColumn='index')                                 
fig, ax = fn.plotTransmission(esM, 'AC cables', transFilePath, loc0='bus0', loc1='bus1', fig=fig, ax=ax)

In [ ]:
transFilePath = os.path.join(cwd, 'InputData', 'SpatialData','ShapeFiles', 'DClines.shp')

fig, ax = fn.plotLocations(locFilePath, indexColumn='index')                                 
fig, ax = fn.plotTransmission(esM, 'DC cables', transFilePath, loc0='cluster0', loc1='cluster1', fig=fig, ax=ax)

In [ ]:
transFilePath = os.path.join(cwd, 'InputData', 'SpatialData','ShapeFiles', 'transmissionPipeline.shp')

fig, ax = fn.plotLocations(locFilePath, indexColumn='index')                                 
fig, ax = fn.plotTransmission(esM, 'Pipelines (hydrogen)', transFilePath, loc0='cluster0', loc1='cluster1',
                              fig=fig, ax=ax)

In [ ]:
transFilePath = os.path.join(cwd, 'InputData', 'SpatialData','ShapeFiles', 'transmissionPipeline.shp')

fig, ax = fn.plotLocations(locFilePath, indexColumn='index')                                 
fig, ax = fn.plotTransmission(esM, 'Pipelines (biogas)', transFilePath, loc0='cluster0', loc1='cluster1',
                              fig=fig, ax=ax)